In [1]:
#install
#@markdown To set up Simple Stable, click on the triangle play button on the left. The interface will appear below.
#@markdown
#@markdown To hide large or annoying code on the left, double-click on this text.
#@markdown
#@markdown To open the interface in a new tab or window, click on the link after "Running on public URL".\
#@markdown When loading a new model for the first time, it may take a few minutes to download, but switching between already downloaded models will be faster.\
#@markdown Please note that time estimates may not be accurate.

starting_model_to_load = "Stable Diffusion 1.5" #@param ["Stable Diffusion 1.4","Stable Diffusion 1.5","Stable Diffusion 2.0 (512)","Stable Diffusion 2.0 (768)","Stable Diffusion 2.1 (512)","Stable Diffusion 2.1 (768)","Pokemon Diffusion","Waifu Diffusion 1.3", "Waifu Diffusion 1.4 (Epoch 1 Test Version)", "AnimeChanStyle", "Anything v3.0","seek.art_MEGA","Dreamlike Diffusion","Sygil Diffusion","Robo Diffusion","Inkpunk Diffusion","Vivid Watercolors","Classic Animation Diffusion","Modern Animation Diffusion","Ghibli Diffusion","Van Gogh Diffusion","Openjourney", "Analog Diffusion", "Future Diffusion","Gigafractal Diffusion"] {type:"string"}

download_a_collection_of_text_inversion_embeddings_and_load_them = False #@param{type: "boolean"}

attention_slicing = False

import os

try:
  import torch
  from src import SimpleStable
except ImportError as e:
  print("Installing required libraries...")
  !pip3 install torch torchvision torchaudio diffusers transformers accelerate scipy pillow tqdm requests huggingface_hub safetensors ipywidgets lark gradio omegaconf --extra-index-url https://download.pytorch.org/whl/cu116 > /dev/null
  if os.name == "nt": #if windows
    !pip install https://github.com/C43H66N12O12S2/stable-diffusion-webui/releases/download/f/xformers-0.0.14.dev0-cp310-cp310-win_amd64.whl
  else:
    ...
  %cd /content/
  !git clone https://github.com/cadaeix/simplest-stable.git > /dev/null
  %cd /content/simplest-stable/
  
%cd /content/simplest-stable/
from datetime import datetime
from IPython.display import display, clear_output
import random
import gradio as gr
import gradio.routes
from src import SimpleStable, utils
import gc
import torch

try:
  outputs_folder
  if not os.path.exists(outputs_folder):
      os.mkdirs(outputs_folder)
  session_folder = outputs_folder
except NameError as e:
  outputs_path = "images/"
  if not os.path.exists(outputs_path):
      os.mkdir(outputs_path)
      
  session_folder = os.path.join(outputs_path, datetime.now().strftime("%Y_%m_%d"))
  if not os.path.exists(session_folder):
      os.mkdir(session_folder)

try:
  custom_models_folder
  custom_models_path = custom_models_folder if os.path.exists(custom_models_folder) else None
except NameError as e:
  custom_models_path = None

try:
  custom_embeddings_folder
  embeddings_path = custom_embeddings_folder if os.path.exists(custom_embeddings_folder) else None
except NameError as e:
  embeddings_path = None

downloaded_embeds = "/content/embeds/"
if download_a_collection_of_text_inversion_embeddings_and_load_them:
  if not os.path.exists(downloaded_embeds):
    !mkdir -p $downloaded_embeds
    %cd /content/embeds/
    !wget https://huggingface.co/cadaeic/v2_dreamink/resolve/main/v2_dreamink.pt
    !wget -O 'v1_bad_prompt.pt' https://huggingface.co/datasets/Nerfgun3/bad_prompt/resolve/main/bad_prompt_version2.pt
    !wget -O 'v2_knollingcase.pt' https://huggingface.co/ProGamerGov/knollingcase-embeddings-sd-v2-0/resolve/main/kc32-v4-5000.pt
    !wget -O 'v2_PhotoHelper.pt' https://huggingface.co/spaablauw/PhotoHelper/resolve/main/PhotoHelper.pt
    !wget -O 'v2_CinemaHelper.pt' https://huggingface.co/spaablauw/CinemaHelper/resolve/main/CinemaHelper.pt
    !wget -O 'v2_glitch.pt' https://huggingface.co/joachimsallstrom/Glitch-Embedding/resolve/main/glitch.pt
    !wget -O 'v1_midj_anthro.pt' https://huggingface.co/crumb/midjourney-textual-inversions/resolve/main/midj-anthro.pt
    !wget -O 'v1_midj_portrait.pt' https://huggingface.co/crumb/midjourney-textual-inversions/resolve/main/midj-portrait.pt
    !wget -O 'v1_midj_strong.pt' https://huggingface.co/crumb/midjourney-textual-inversions/resolve/main/midj-strong.pt
    !wget -O 'v2_wrong.bin' https://huggingface.co/minimaxir/wrong_embedding_sd_2_0/resolve/main/learned_embeds.bin
    !wget -O 'v2_laxpeint.pt' https://huggingface.co/EldritchAdam/laxpeint/resolve/main/laxpeintV2.pt
    !wget -O 'v2_classipeint.pt' https://huggingface.co/EldritchAdam/classipeint/blob/main/classipeint.pt
    !wget -O 'v2_AnimeScreencap.pt' https://huggingface.co/Conflictx/AnimeScreencap/resolve/main/AnimeScreenCap.pt
    !wget -O 'v2_DoubleExposure.pt' https://huggingface.co/joachimsallstrom/Double-Exposure-Embedding/resolve/main/dblx.pt
    !wget -O 'v1_charturner' https://huggingface.co/Mousewrites/charTurner/resolve/main/charTurner.pt

%cd /content/simplest-stable/
print(f"Saving images to {session_folder}.")
if custom_models_path:
  print(f"Found custom models folder at {custom_models_path}!")
if embeddings_path:
  print(f"Found custom embeddings folder at {embeddings_path}!")

print("Loading models and files...")

css = ""

with open("src/gradio.css") as file:
    css += file.read() + "\n"

pipe, pipe_info = SimpleStable.load_downloadable_model(starting_model_to_load, None)
if attention_slicing:
    pipe.enable_attention_slicing()

def load_embeddings_from_folders():
  global pipe
  if embeddings_path:
    SimpleStable.load_embeddings(embeddings_path, pipe)
  if os.path.exists(downloaded_embeds):
    SimpleStable.load_embeddings(downloaded_embeds, pipe)

load_embeddings_from_folders()
pipe.to("cuda")

all_custom_models = utils.find_custom_models(custom_models_path)
all_cached_hf_models = utils.get_all_cached_hf_models(all_custom_models)

clear_output(wait=False)

class LoadJavaScript():
    def __init__(self):
        self.original_template = gradio.routes.templates.TemplateResponse

        with open("src/gradio.js", "r", encoding="utf8") as jsfile:
            self.javascript = f'<script>{jsfile.read()}</script>'
        
        gradio.routes.templates.TemplateResponse = self.template_response

    def template_response(self, *args, **kwargs):
        response = self.original_template(*args, **kwargs)
        response.body = response.body.replace(
            '</head>'.encode('utf-8'), f"{''.join(self.javascript)}\n</head>".encode("utf-8")
        )
        response.init_headers()
        return response

def return_selected_image_from_gallery(i):
    return i["name"] if i else None

def update_model_lists():
    global all_custom_models, all_cached_hf_models
    all_custom_models = utils.find_custom_models(custom_models_path)
    all_cached_hf_models = utils.get_all_cached_hf_models(all_custom_models)


def load_a_model(loaded_model_name: str, chosen_model_name: str, load_model_function, progress=gr.Progress(track_tqdm=True)):
    global pipe, pipe_info

    if pipe is None or loaded_model_name != chosen_model_name:
        del pipe
        gc.collect()
        torch.cuda.empty_cache()
        pipe, pipe_info = load_model_function(chosen_model_name, all_custom_models, all_cached_hf_models)
        pipe.to("cuda")
        if attention_slicing:
           pipe.enable_attention_slicing()
        load_embeddings_from_folders()
        update_model_lists()
        return f"{chosen_model_name} loaded", chosen_model_name

    return "Model already loaded", loaded_model_name

def choose_type_and_load_model(dropdown_type: str, loaded_model_name: str, downloadable_model_name: str, cached_model_name: str, custom_model_name: str, progress=gr.Progress(track_tqdm=True)):
    global pipe, pipe_info, all_custom_models

    if dropdown_type == "Installed Models" and cached_model_name:
        return load_a_model(loaded_model_name, cached_model_name, SimpleStable.load_cached_model)
    elif dropdown_type ==  "Downloadable Models" and downloadable_model_name:
        return load_a_model(loaded_model_name, downloadable_model_name, SimpleStable.load_downloadable_model)
    elif dropdown_type == "Custom Models" and custom_model_name: 
        if custom_model_name in all_cached_hf_models:
            return load_a_model(loaded_model_name, custom_model_name, SimpleStable.load_cached_model)
        else:
            return load_a_model(loaded_model_name, custom_model_name, SimpleStable.load_custom_model)
    else:
        return "Choice not valid", loaded_model_name

def is_custom_resolution(resolution: str):
    return resolution == "Custom (Select this and put width and height below)"

def generate(mode, prompt, negative, number_of_images, resolution, custom_width, custom_height, steps, sampler, seed, scale, additional_options, upscale_strength, input_image, img2img_strength, inpaint_image, inpaint_strength, model_name, progress=gr.Progress(track_tqdm=True)):
    global pipe, pipe_info, session_folder

    if mode == "txt2img":
        init_img = None
        mask_image = None
        strength = None
    elif mode == "img2img":
        init_img = input_image
        mask_image = None
        strength = img2img_strength
    elif mode == "inpainting":
        init_img = inpaint_image["image"]
        mask_image = inpaint_image["mask"]
        strength = inpaint_strength

    negative = negative if negative != None else ""
    used_seed = random.randint(0, 2**32) if seed < 0 else seed
    width, height = [custom_width, custom_height] if is_custom_resolution(resolution) else SimpleStable.res_dict[resolution]
 
    if "Insert standard Danbooru model quality prompt" in additional_options:
        prompt = "masterpiece, best quality, " + prompt
        standard_negative = "lowres, bad anatomy, bad hands, text, error, missing fingers, extra digit, fewer digits, cropped, worst quality, low quality, normal quality, jpeg artifacts, signature, watermark, username"
        negative = standard_negative if negative is None else standard_negative + ", " + negative

    images, settings_info = SimpleStable.gradio_main({
        "model_name": model_name,
        "prompt": prompt,
        "negative": negative if negative != None else "",
        "init_img": init_img,
        "mask_image": mask_image,
        "strength": strength,
        "number_of_images": number_of_images,
        "H" : height - height % 64,
        "W" : width - width % 64,
        "steps": steps,
        "sampler": sampler,
        "scale": scale,
        "eta" : 0.0,
        "tiling" : "Tiling" in additional_options,
        "upscale": "SD Upscale" in additional_options,
        "upscale_strength": upscale_strength if "SD Upscale" in additional_options else None,
        "detail_scale" : 10,
        "seed": used_seed,
        "add_keyword": "Don't insert model keyword" not in additional_options,
        "keyword": pipe_info["keyword"],
        "outputs_folder": session_folder,
        "prediction_type": pipe_info["prediction_type"]
    }, pipe)

    message = '\n\n'.join(settings_info)

    return images, used_seed, message

def generate_options():
    with gr.Row():
        with gr.Column(scale=1):
            number_of_images = gr.Number(value=1, precision=0, label="Number of Images")
        with gr.Column(scale=2):
            resolution = gr.Dropdown(choices = list(SimpleStable.res_dict.keys()), label="Image Resolution", value="Square 512x512 (default, good for most models)")

    with gr.Accordion(label="Advanced Settings", elem_id="adv_settings"):
        with gr.Row():
            with gr.Column():
                custom_width = gr.Slider(minimum = 512, maximum = 1152, value= 512, step = 64, label="Width (if Custom is selected)", interactive = True, elem_id="custom_width")
                sampler = gr.Dropdown(choices = SimpleStable.sampler_list, label="Sampler", value="Euler a", elem_id="sampler_choice")
                with gr.Row():
                    with gr.Column(elem_id="seed_col"):
                        seed = gr.Number(value=-1, precision=0, label="Seed", interactive=True)
                    with gr.Column(elem_id="seed_button_col"):
                        reuse_seed_button = gr.Button(value="Last Seed", elem_id="reuse_seed")
                        random_seed_button = gr.Button(value="Random Seed", elem_id="random_seed")
                additional_options = gr.CheckboxGroup(["Tiling", "SD Upscale", "Don't insert model keyword", "Insert standard Danbooru model quality prompt"], interactive=True, label="Additional Settings")
            with gr.Column():
                custom_height = gr.Slider(minimum = 512, maximum = 1152, value= 512, step = 64, label="Height (if Custom is selected)", interactive = True, elem_id="custom_height")
                steps = gr.Slider(minimum = 1, maximum = 100, value= 20, step = 1, label="Step Count", interactive = True, elem_id="step_count")
                scale = gr.Slider(minimum = 1, maximum = 20, value= 7, step = 0.5, label="Guidance Scale", interactive = True, elem_id="guidance_scale")
                upscale_strength = gr.Slider(minimum = 0.1, maximum = 1, value=0.2, step = 0.05, label="Upscale Strength", interactive = True, elem_id="upscale_strength")
    
    return number_of_images, resolution, custom_width, custom_height, steps, sampler, seed, scale, additional_options, upscale_strength, reuse_seed_button, random_seed_button

def has_image(input):
    return {
        to_img2img_button: gr.update(visible = (input is not None)),
        to_inpaint_button: gr.update(visible = (input is not None)),
        log_output: gr.update(visible = (input is not None)),
    }

def show_state(input: str):
    states = {
        "txt2img": [True, False, False],
        "img2img": [False, True, False],
        "inpainting": [False, False, True]
    }

    txt2img_button = "primary" if states[input][0] else "secondary"
    img2img_button = "primary" if states[input][1] else "secondary" 
    inpaint_button = "primary" if states[input][2] else "secondary"

    return {
        current_mode: input,
        input_image: gr.update(visible = states[input][1]), 
        img2img_strength: gr.update(visible = states[input][1]),
        inpaint_image: gr.update(visible = states[input][2]), 
        inpaint_strength: gr.update(visible = states[input][2]),
        txt2img_show: gr.update(variant = txt2img_button), 
        img2img_show: gr.update(variant = img2img_button),  
        inpaint_show: gr.update(variant = inpaint_button)
        }

def show_state_and_clear_inpaint(input: str):
    states = {
        "txt2img": [True, False, False],
        "img2img": [False, True, False],
        "inpainting": [False, False, True]
    }

    txt2img_button = "primary" if states[input][0] else "secondary"
    img2img_button = "primary" if states[input][1] else "secondary" 
    inpaint_button = "primary" if states[input][2] else "secondary"

    return {
        current_mode: input,
        input_image: gr.update(visible = states[input][1]), 
        img2img_strength: gr.update(visible = states[input][1]),
        inpaint_image: gr.update(value = None, visible = states[input][2]), 
        inpaint_strength: gr.update(visible = states[input][2]),
        txt2img_show: gr.update(variant = txt2img_button), 
        img2img_show: gr.update(variant = img2img_button),  
        inpaint_show: gr.update(variant = inpaint_button)
        }

def update_model_lists_in_gradio():
    return {
        cached_models: gr.update(choices = list(all_cached_hf_models.keys())),
        custom_models: gr.update(choices = list(all_custom_models.keys()))
    }

#run
load_javascript = LoadJavaScript()
with gr.Blocks(css=css, title="Simple Stable") as main:
    current_loaded_model_name = gr.State("Stable Diffusion 1.5")
    current_mode = gr.State("txt2img")
    last_used_seed = gr.State(-1)

    with gr.Row(elem_id="model_row"):
        model_dropdown_type = gr.Dropdown(choices = ["Installed Models", "Downloadable Models", "Custom Models"], show_label=False, elem_id="model_dropdown_type")
        downloadable_models = gr.Dropdown(choices = list(SimpleStable.model_dict.keys()), value = "Stable Diffusion 1.5", show_label=False, elem_id="download_model_choice", interactive=True)
        cached_models = gr.Dropdown(choices = list(all_cached_hf_models.keys()), show_label=False, elem_id="cached_model_choice", interactive=True)
        custom_models = gr.Dropdown(choices = list(all_custom_models.keys()), show_label=False, elem_id="custom_model_choice", interactive=True)
        model_submit = gr.Button(value="Load Model", interactive=True, elem_id="model_submit")
    loading_status = gr.Markdown("", elem_id="model_status")

    with gr.Row():
        with gr.Column(scale=3):
            with gr.Row():
                txt2img_show = gr.Button(value="txt2img", variant="primary")
                img2img_show = gr.Button(value="img2img")
                inpaint_show = gr.Button(value="inpainting")

            prompt = gr.Textbox(placeholder = "Describe a prompt here", label = "Prompt")
            negative = gr.Textbox(placeholder = "Negative prompt", label = "Negative")

            input_image = gr.Image(value=None, source="upload", interactive=True, type="pil", visible=False, elem_id="img2img_input")
            inpaint_image = gr.Image(value=None, source="upload", interactive=True, type="pil", visible=False, tool="sketch", elem_id="inpaint_input")
            img2img_strength = gr.Slider(minimum = 0.1, maximum = 1, value=0.75, step = 0.05, label="img2img strength", interactive = True, visible=False, elem_id="img2img_strength")
            inpaint_strength = gr.Slider(minimum = 0.1, maximum = 1, value=0.75, step = 0.05, label="inpaint strength", interactive = True, visible=False, elem_id="inpaint_strength")
            # inpaint_modes = gr.Dropdown(value="Inpaint Over Original Picture (good for editing)", choices=["Inpaint Over Original Picture (good for editing)", "Inpaint over Latent Noise (better for adding new things)"], visible=False, interactive = True, elem_id="inpaint_modes")

            number_of_images, resolution, custom_width, custom_height, steps, sampler, seed, scale, additional_options, upscale_strength, reuse_seed_button, random_seed_button = generate_options()

        with gr.Column(scale=2):
            with gr.Box(elem_id="output_box") as output_box:
                with gr.Row(elem_id="generate_row"):
                    generate_button = gr.Button(value="Generate", variant="primary", elem_id="generate_button")
                    #interrupt_button = gr.Button(value="Interrupt", variant="secondary", elem_id="generate_button")
                image_output = gr.Gallery(elem_id="output_gallery")
                with gr.Row(elem_id="edit_row"):
                    to_img2img_button = gr.Button(value="img2img Selected Image", variant="secondary", elem_id="to_img2img_button", visible=False)
                    to_inpaint_button = gr.Button(value="inpaint Selected Image", variant="secondary", elem_id="to_inpaint_button", visible=False)
                log_output = gr.Textbox(interactive = False, elem_id = "log_output", show_label=False, visible=False, lines=7)

    model_submit.click(choose_type_and_load_model, inputs=[model_dropdown_type, current_loaded_model_name, downloadable_models, cached_models, custom_models], outputs=[loading_status, current_loaded_model_name], preprocess=False, postprocess=False)
    model_submit.click(update_model_lists_in_gradio, inputs=[], outputs=[cached_models, custom_models])

    reuse_seed_button.click(lambda x: x, inputs=[last_used_seed], outputs=[seed], preprocess=False, postprocess=False)
    random_seed_button.click(lambda : -1, inputs=[], outputs=[seed], preprocess=False, postprocess=False)

    txt2img_show.click(show_state, inputs=[txt2img_show], outputs = [current_mode, input_image, img2img_strength, inpaint_image, inpaint_strength, txt2img_show, img2img_show, inpaint_show], preprocess=False, postprocess=False)
    img2img_show.click(show_state, inputs=[img2img_show], outputs = [current_mode, input_image, img2img_strength, inpaint_image, inpaint_strength, txt2img_show, img2img_show, inpaint_show], preprocess=False, postprocess=False)
    inpaint_show.click(show_state, inputs=[inpaint_show], outputs = [current_mode, input_image, img2img_strength, inpaint_image, inpaint_strength, txt2img_show, img2img_show, inpaint_show], preprocess=False, postprocess=False)

    to_img2img_button.click(show_state, inputs=[img2img_show], outputs = [current_mode, input_image, img2img_strength, inpaint_image, inpaint_strength, txt2img_show, img2img_show, inpaint_show], preprocess=False, postprocess=False)
    to_inpaint_button.click(show_state_and_clear_inpaint, inputs=[inpaint_show], outputs = [current_mode, input_image, img2img_strength, inpaint_image, inpaint_strength, txt2img_show, img2img_show, inpaint_show], preprocess=False, postprocess=False)
    to_img2img_button.click(return_selected_image_from_gallery, inputs=[image_output], outputs =[input_image], _js="findSelectedImageFromGallery")
    to_inpaint_button.click(return_selected_image_from_gallery, inputs=[image_output], outputs =[inpaint_image], _js="findSelectedImageFromGallery")
    generate_button.click(generate, inputs=[current_mode, prompt, negative, number_of_images, resolution, custom_width, custom_height, steps, sampler, seed, scale, additional_options, upscale_strength, input_image, img2img_strength, inpaint_image, inpaint_strength, current_loaded_model_name], outputs=[image_output, last_used_seed, log_output])
    generate_button.click(has_image, inputs=[image_output], outputs=[to_img2img_button, to_inpaint_button, log_output])
    #hidden_state.change(lambda: "", inputs=[], outputs=[hidden_state], _js="setupOutputGallery")

    #model_dropdown_type.change(handle_model_dropdowns, inputs = [model_dropdown_type], outputs=[downloadable_models, cached_models, custom_models])
    model_dropdown_type.change(None, _js="handleModelDropdowns", inputs=[], outputs=[])

main.queue()
app, local_url, share_url = main.launch(debug=True, inline=True, height=1000)

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
